# 扩增数据

在shuffle-data中，数据扩增在train-dev上面，test数据用于做模型评估用，所以不进行扩增。

扩增规则：

> if A==B & A==C  then B==C 

只扩增正样本，负样本暂不进行处理

## 筛选并按照query1进行group分组，整理出list

In [2]:
train_data = pd.read_csv("shuffle-data/train_data.csv")
dev_data = pd.read_csv("shuffle-data/dev_data.csv")
train = pd.concat([train_data,dev_data])
train

,category,query1,query2,label
0,咳血,剧烈运动后咯血，是怎么了？,剧烈运动后咯血是什么原因？,1
1,咳血,剧烈运动后咯血，是怎么了？,剧烈运动后为什么会咯血？,1
2,咳血,剧烈运动后咯血，是怎么了？,剧烈运动后咯血，应该怎么处理？,0
3,咳血,剧烈运动后咯血，是怎么了？,剧烈运动后咯血，需要就医吗？,0
4,咳血,剧烈运动后咯血，是怎么了？,剧烈运动后咯血，是否很严重？,0
...,...,...,...,...
1396,肺炎,请问有肺炎该怎么治疗？,肺炎需要吸痰吗,0
1397,支原体肺炎,得了小儿支原体肺炎应该注意什么？,小儿支原体肺炎有什么需要注意的？,1
1398,哮喘,急性支气管炎治疗方法？,急性支气管炎的治疗方法是什么？,1
1399,感冒,两种感冒药混吃会要命吗,两种感冒药混吃可以吗,1


In [3]:
def get_query_list(data_group):
    l = []
    for query in data_group:
        l.append(query)
    return l

new = train.loc[train["label"]==1,:].groupby("query1")["query2"].agg(get_query_list)

In [4]:
new.values

array([list(['加湿器肺炎是怎样回事']),
       list(['11个月大孩子老是流鼻涕怎么治', '宝宝11个月了，老是流鼻涕有什么办法']),
       list(['肺炎13介疫苗需要打吗？', '有必要打13介肺炎疫苗吗？', '打13介肺炎疫苗管用吗？', '注射13介肺炎疫苗有作用吗？']),
       ..., list(['鼻疽假单胞菌肺炎有怎样的治疗办法？', '鼻疽假单胞菌肺炎的治疗方法是什么？']),
       list(['鼻子有血丝出来是鼻咽癌吗？', '回吸鼻涕有血丝，是鼻咽癌吗？', '鼻痛鼻涕有血丝是不是鼻癌', '鼻痛鼻涕有血丝跟鼻癌有什么关系吗']),
       list(['吃了太多的龙鹿丸导致嗓子疼该怎么办', '龙鹿丸吃了嗓子疼是什么情况'])], dtype=object)

## 在list中两两组合成pair

In [5]:
new_data = pd.DataFrame(columns=('category', 'query1', 'query2','label'))
for line in new:
    if len(line)>1:
        content = line[0]
        category = train.loc[train["query2"]==content,"category"].values[0]
        for idx1,q1 in enumerate(line):
            for idx2 in range(idx1+1,len(line)):
                query1 = line[idx1]
                query2 = line[idx2]
                new_data = new_data.append([{'category':category,'query1':query1,'query2':query2,'label':1}])

In [6]:
new_data

,category,query1,query2,label
0,感冒,11个月大孩子老是流鼻涕怎么治,宝宝11个月了，老是流鼻涕有什么办法,1
0,肺炎,肺炎13介疫苗需要打吗？,有必要打13介肺炎疫苗吗？,1
0,肺炎,肺炎13介疫苗需要打吗？,打13介肺炎疫苗管用吗？,1
0,肺炎,肺炎13介疫苗需要打吗？,注射13介肺炎疫苗有作用吗？,1
0,肺炎,有必要打13介肺炎疫苗吗？,打13介肺炎疫苗管用吗？,1
...,...,...,...,...
0,感冒,鼻子有血丝出来是鼻咽癌吗？,鼻痛鼻涕有血丝跟鼻癌有什么关系吗,1
0,感冒,回吸鼻涕有血丝，是鼻咽癌吗？,鼻痛鼻涕有血丝是不是鼻癌,1
0,感冒,回吸鼻涕有血丝，是鼻咽癌吗？,鼻痛鼻涕有血丝跟鼻癌有什么关系吗,1
0,感冒,鼻痛鼻涕有血丝是不是鼻癌,鼻痛鼻涕有血丝跟鼻癌有什么关系吗,1


可以看到，按照上述规则，可以扩增了3251条数据（无序，即不区分A-B与B-A）

# 重新再切新的train和dev

In [7]:
new_train = pd.concat([train,new_data])
new_train = new_train.reset_index()
new_train = new_train.drop(labels="index",axis = 1)
new_train

,category,query1,query2,label
0,咳血,剧烈运动后咯血，是怎么了？,剧烈运动后咯血是什么原因？,1
1,咳血,剧烈运动后咯血，是怎么了？,剧烈运动后为什么会咯血？,1
2,咳血,剧烈运动后咯血，是怎么了？,剧烈运动后咯血，应该怎么处理？,0
3,咳血,剧烈运动后咯血，是怎么了？,剧烈运动后咯血，需要就医吗？,0
4,咳血,剧烈运动后咯血，是怎么了？,剧烈运动后咯血，是否很严重？,0
...,...,...,...,...
13394,感冒,鼻子有血丝出来是鼻咽癌吗？,鼻痛鼻涕有血丝跟鼻癌有什么关系吗,1
13395,感冒,回吸鼻涕有血丝，是鼻咽癌吗？,鼻痛鼻涕有血丝是不是鼻癌,1
13396,感冒,回吸鼻涕有血丝，是鼻咽癌吗？,鼻痛鼻涕有血丝跟鼻癌有什么关系吗,1
13397,感冒,鼻痛鼻涕有血丝是不是鼻癌,鼻痛鼻涕有血丝跟鼻癌有什么关系吗,1


In [8]:
from sklearn.model_selection import train_test_split
train_index,dev_index,_,_ = train_test_split(new_train.index,new_train["label"],
                                             test_size=0.2,random_state=2020,
                                             stratify=new_train["label"])
train_data = new_train.iloc[train_index,:]
dev_data = new_train.iloc[dev_index,:]

In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10719 entries, 3519 to 3091
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  10719 non-null  object
 1   query1    10719 non-null  object
 2   query2    10719 non-null  object
 3   label     10719 non-null  object
dtypes: object(4)
memory usage: 418.7+ KB


In [10]:
dev_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2680 entries, 12486 to 1440
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2680 non-null   object
 1   query1    2680 non-null   object
 2   query2    2680 non-null   object
 3   label     2680 non-null   object
dtypes: object(4)
memory usage: 104.7+ KB


In [11]:
train_data.to_csv("augment-data/train_data.csv",index=False)
dev_data.to_csv("augment-data/dev_data.csv",index=False)